# Tech 2025 Lab
### Lab A - Loan Risk AI Agent
An **AI Agent** refers to a system or program that is capable completing a request by first planning the sequence of workflow steps and then performing the tasks in the steps by utilizing available tools. AI agents utilizes large language models (LLM) to understand the context of the request, plan and perform the tasks. 

This Python Notebook provides the code to create a simple AI agent for risk and interest rate evaluation in a bank loan processing scenario. It uses LangGraph graph for state and runtime processing. Once created, you assume the persona of a loan risk analyst at a bank and ask the Loan Risk AI Agent questions in natural language to assess risks for customers. When a question is asked, the AI agent will use a LLM to understand the context, determine the seqence of steps, and then complete the steps utilizing availble tools. The tools fetch customer information, credit score, account status and determine risk and interest rate. The sequence of steps, tool calling and final response can be reviewed in the output.

#### To use this Notebook:

Run each cell below one by one and make sure it completes successfully.

When prompted for API key enter it in the box and hit Enter and then continue running the cells to initialize the code.

Finally, when prompted enter your query and run the subsequent cells. Some example questions are provided.

You may run/repeat the query cells by entering and trying different queries.

#### Lab A - Exercise 1
Below is an outline of the code. The code is set up in Python Notebook cells.

1. Set up required libraries
2. Define functions to get and check credentials
3. Define tools that the AI agent can use
4. Configure the LLM
5. Define the LangGraph graph and functions for state and runtime processing.
6. Show a visual representation the graph - the AI agent with tools
7. Use the AI agent - ask the AI agent risk related question
8. Review the responses from AI agent risk

#### Lab A - Exercise 2
1. Add another tool that can provide interest rate.
2. Use the AI agent - ask the AI agent risk and interest rate related question
3. Review the responses from AI agent risk



### Lab A - Exercise 1

#### 1. Set up required libraries

In [ ]:
# Set up libraries
%pip install langgraph==0.2.73
%pip install -U langchain-ibm==0.3.6
    
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage

from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

import requests
import json
import time
import random

#### 2. Define functions to get and check credentials

In [ ]:
# Function to get credentials
import getpass
import os

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

# Get credentials
# Set the Project IDs
# If using you own account, or lab account and project, the current project id will be configured and set automtically
_set_if_undefined("PROJECT_ID")

# When prompted for API key enter it in the box and hit Enter, move to next cell and then continue running the cells
_set_if_undefined("WATSONX_API_KEY")


In [ ]:
# Print credential configurations for validation
print('Api key:',os.environ.get("WATSONX_API_KEY"))
print('Project id:',os.environ.get("PROJECT_ID"))

#### 3. Define tools that the AI agent can use

In [ ]:
# Define tools that the AI agent can use

# The tool get_customer_info() simulates/mocks API call that fetches credit score and account status information about a customer
# The tools get_credit_score() and get_account_status() use the customer info tool to get the score and account status
# The customer's loan risk is determined based on credit score and account status risk rules using get_overall_risk() tool
# When a natural language question is asked, the AI agent first determines the context and then the sequence to utilize one or more tools to provide a response 

# this tool simulates an API call that will fetch information about customers
@tool
def get_customer_info(customer_id: str): 
    """Retrieve customer information based on the customer_id."""
    customer_id = customer_id.lower()
    
    if customer_id in ['loren@ibm.com', 'loren', '1111']:
        return {'credit_score': 455, 'account_status': 'good-standing'}
    elif customer_id in ['matt@ibm.com', 'matt', '2222']:
        return {'credit_score': 685, 'account_status': 'closed'}
    elif customer_id in ['hilda@ibm.com', 'hilda', '3333']:
        return {'credit_score': 825, 'account_status': 'delinquent'}
    else:
        #return {'credit_score': None, 'account_status': None}
        #if any other customer id is used, provide a random credit score and account status
        return {'credit_score': random.randint(300, 850), 'account_status': random.choice(['delinquent', 'good-standing', 'closed' ])}



# this tool fetches credit score of a customer
@tool
def get_credit_score(customer_id: str) -> int: 
    """Get the credit score for the customer using customer_id. Customer's name can be used instead of customer_id."""
    customer_info = get_customer_info(customer_id)
    return customer_info['credit_score']
    customer_id = customer_id.lower()


# this tool fetches account status of a customer
@tool
def get_account_status(customer_id: str) -> str: 
    """Get the account status for the customer using customer_id. Customer's name can be used instead of customer_id."""
    customer_info = get_customer_info(customer_id)
    return customer_info['account_status']
    customer_id = customer_id.lower()

# this tool determines the overall risk for a customer
@tool
def get_overall_risk(credit_score: int, account_status: str):
    """Get overall risk based on combination of both credit score and account status. Only use high, medium or low as risk categories. Explain how the overall risk was calculated. If the credit score and account status are unknown then do not provide the risk status and first retrieve the missing credit score or account status."""

    #print("get_overall_risk():credit score:account status::", credit_score, account_status)
    
    if (credit_score > 299 & credit_score < 674): 
        if (account_status == 'delinquent'): overall_risk = 'high'
        elif (account_status == 'closed'):  overall_risk = 'high'
        elif (account_status == 'good-standing'):  overall_risk = 'medium'
        else: overall_risk = 'unkown'
    
    elif (credit_score >= 675 & credit_score < 749): 
        if (account_status == 'delinquent'): overall_risk = 'high'
        elif (account_status == 'closed'):  overall_risk = 'medium'
        elif (account_status == 'good-standing'):  overall_risk = 'medium'
        else: overall_risk = 'unkown'

    elif (credit_score >= 750 & credit_score < 851): 
        if (account_status == 'delinquent'): overall_risk = 'high'
        elif (account_status == 'closed'):  overall_risk = 'low'
        elif (account_status == 'good-standing'):  overall_risk = 'low'
        else: overall_risk = 'unkown'
    

    else: overall_risk = 'unable to determine.'    
    return overall_risk

# Set the list of tools to be used by the AI agent
tools = [get_credit_score, get_account_status, get_overall_risk, get_customer_info]

tool_node = ToolNode(tools)



#### 4. Configure the LLM

In [ ]:
#Configure the LLM

from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "sample", #"greedy", #"sample"
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 150,
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 250,
    GenTextParamsMetaNames.TEMPERATURE: 0,
    #GenTextParamsMetaNames.TOP_K: 50,
    #GenTextParamsMetaNames.TOP_P: 1,
}

from langchain_ibm import ChatWatsonx
model = ChatWatsonx(
    model_id="mistralai/mistral-large", 
    #url="https://us-south.ml.cloud.ibm.com", 
    url="https://eu-de.ml.cloud.ibm.com", 
    apikey=os.environ.get("WATSONX_API_KEY"),
    project_id=os.environ.get("PROJECT_ID"),
    params=parameters,
).bind_tools(tools)

#### 5. Define the LangGraph graph and functions for state and runtime processing.

In [ ]:
# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}



In [ ]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# add a normal edge from `tools` to `agent`.
# after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')


# Compile graph
app = workflow.compile()


#### 6. Show a visual representation the graph - the AI agent with tools

In [ ]:
# Show graph
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

#### 7. Use the AI agent - ask the AI agent risk related question



Ask the AI agent these questions. Review the response.

- what can you do for me?
- what can you do for me? how?

- what is the risk for matt?
 
- what is the risk for matt? explain why?

- tell me about hilda?
  
- what is the risk for hilda? explain why?

- what is the risk for credit score 774 and account status delinquent?

- what is the risk for credit score 774 and account status delinquent? how was it determined?

Also try asking questions about interest rate....

- what is the interest rate for matt?

- what is the interest rate for matt? explain how it was determined?

For the last two questions, note that there is no tool yet that provides interest rate. So the response does not answer the question appropriately. 
We will add interest rate tool in the next exercise.

#### Write the query in the box and hit Enter and then continue running the next cells

In [ ]:
human_query = input("Type a question and hit enter: ")

In [ ]:
# Use the runtime
final_state = app.invoke(
    #{"messages": [HumanMessage(content="what is the overall risk?")]},
    {"messages": [HumanMessage(content=human_query)]},
    config={"configurable": {"thread_id": random.randint(1, 100)}}
)

#final_state["messages"][-1].content

for m in final_state["messages"]:
    m.pretty_print()

#Print final message
#final_state["messages"][-1].content


#### 8. Review the response above from AI agent risk
Run/repeat the above cells with different queries, i.e., rerun the human_query and final_state = app.invoke() cells with different queries

<a id='lab_a_exercise_2'></a>
### Lab A - Exercise 2


So far we had tool that provided the risk level for a customer - as low, medium or high. There was no tool to provide a specific interest rate.
In this exercise we will add another tool that can provide the interest rate based on the risk level.


In [ ]:
# this tool determines the interest rate for a customer
@tool
def get_interest_rate(overall_risk: str):
    """Get interest rate percentage based on the overall risk. If the overall risk is not known then do not provide the interest rate and first retrieve the missing overall risk."""
    if (overall_risk.lower() == "high"):
        interest_rate=10.75;
    
    elif (overall_risk.lower() == "medium"):
        interest_rate=5.25;

    elif (overall_risk.lower() == "low"):
        interest_rate=3.0;
    
    else: interest_rate = 'unable to determine.'    
    return interest_rate


# Lets add/include this tool in the list of tools used by the AI agent in the following code line "tools =..."
tools = [get_credit_score, get_account_status, get_overall_risk, get_interest_rate, get_customer_info]
tool_node = ToolNode(tools)

In [ ]:
# Re-bind the model configuration with new tools

from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "sample", #"greedy", #"sample"
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 150,
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 250,
    GenTextParamsMetaNames.TEMPERATURE: 0,
    #GenTextParamsMetaNames.TOP_K: 50,
    #GenTextParamsMetaNames.TOP_P: 1,
}

from langchain_ibm import ChatWatsonx
model = ChatWatsonx(
    model_id="mistralai/mistral-large", 
    #url="https://us-south.ml.cloud.ibm.com", 
    url="https://eu-de.ml.cloud.ibm.com", 
    apikey=os.environ.get("WATSONX_API_KEY"),
    project_id=os.environ.get("PROJECT_ID"),
    params=parameters,
).bind_tools(tools)

In [ ]:
# Re-define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Re-define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
# Re-define a new graph with new tools
workflow = StateGraph(MessagesState)

# Re-define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# add a normal edge from `tools` to `agent`.
# after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')


# Re-compile graph
app = workflow.compile()



Ask the AI agent questions about interest rate. Review the response.

 
- what is the interest rate for matt?

- what is the interest rate for matt? explain how it was determined?

Note that how the AI agent can now use the get_interest_rate to provide interest rate also.

#### Write the query in the box and hit Enter and then continue running the next cells

In [ ]:
human_query = input("Type a question and hit enter: ")


In [ ]:
# Use the runtime
final_state = app.invoke(
    #{"messages": [HumanMessage(content="what is the overall risk?")]},
    {"messages": [HumanMessage(content=human_query)]},
    config={"configurable": {"thread_id": random.randint(1, 100)}}
)

#final_state["messages"][-1].content

for m in final_state["messages"]:
    m.pretty_print()

#Print final message
#final_state["messages"][-1].content